In [1]:
from os import chdir, path, getcwd
for i in range(10):
    if path.isfile('checkcwd'):
        break
    chdir(path.pardir)
if path.isfile('checkcwd'):
    pass
else:
    raise Exception('Something went wrong. cwd=' + getcwd())

In [2]:
import pandas as pd
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn import preprocessing
import wandb
import datetime
from tqdm import tqdm
from collections import defaultdict
import joblib

In [3]:
RATINGS_DIR = 'resources/data/train_val_test'
CHECKPOINT_PATH = 'resources/checkpoints'

batch_size = 512
layers = [32, 16, 8]
reg_layers = [0.3, 0.3, 0.3]
learning_rate = 5e-4
epochs = 40

k = 25
threshold = 6

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
df_train = pd.read_csv(f'{RATINGS_DIR}/ratings_train.csv', header=None, dtype='int32')
df_train.columns = ['uid', 'fid', 'rating']

In [5]:
df_val = pd.read_csv(f'{RATINGS_DIR}/ratings_val.csv', header=None, dtype='int32')
df_val.columns = ['uid', 'fid', 'rating']

In [6]:
# df = pd.concat([df_train, df_val], ignore_index=True) # no effect

In [7]:
# from sklearn.model_selection import train_test_split
# df_train, df_val = train_test_split(
#     df, test_size=0.1111, stratify=df.rating.values
# )

In [8]:
class MyDataset(Dataset):
    def __init__(self, users, movies, ratings):
        self.users = users
        self.movies = movies
        self.ratings = ratings

    def __len__(self):
        return len(self.users)

    def __getitem__(self, item):
        users = self.users[item]
        movies = self.movies[item]
        ratings = self.ratings[item]

        return {
            'uid': torch.tensor(users, dtype=torch.int),
            'fid': torch.tensor(movies, dtype=torch.int),
            'rating': torch.tensor(ratings, dtype=torch.float),
        }

In [9]:
label_enc_uid = preprocessing.LabelEncoder()
label_enc_fid = preprocessing.LabelEncoder()

# label_enc_fid.fit(df.fid.values)
# label_enc_uid.fit(df.uid.values)   

# df_train.uid = label_enc_uid.transform(df_train.uid.values)
# df_train.fid = label_enc_fid.transform(df_train.fid.values)

df_train.uid = label_enc_uid.fit_transform(df_train.uid.values)
df_train.fid = label_enc_fid.fit_transform(df_train.fid.values)

df_val.uid = label_enc_uid.transform(df_val.uid.values)
df_val.fid = label_enc_fid.transform(df_val.fid.values)

joblib.dump(label_enc_fid, f'{CHECKPOINT_PATH}/label_enc_fid.pkl')
joblib.dump(label_enc_uid, f'{CHECKPOINT_PATH}/label_enc_uid.pkl')

['resources/checkpoints/label_enc_uid.pkl']

In [10]:
train_dataset = MyDataset(
    list(df_train.uid),
    list(df_train.fid),
    list(df_train.rating)
)

val_dataset = MyDataset(
    list(df_val.uid),
    list(df_val.fid),
    list(df_val.rating)
)

# r = 4

# train_dataset = MyDataset(
#     list(df_train.uid)[::r],
#     list(df_train.fid)[::r],
#     list(df_train.rating)[::r]
# )

# val_dataset = MyDataset(
#     list(df_val.uid)[::r],
#     list(df_val.fid)[::r],
#     list(df_val.rating)[::r]
# )

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    drop_last=True
)

In [12]:
class RecSysModel(nn.Module):
    def __init__(self, num_users, num_items, layers, reg_layers):
        super(RecSysModel, self).__init__()
        self.num_layers = len(layers)

        # split for concat later
        self.user_embedding = nn.Embedding(num_users, layers[0] // 2)
        self.item_embedding = nn.Embedding(num_items, layers[0] // 2)

        fc_layers = []
        input_size = layers[0]
        for i in range(1, len(layers)):
            fc_layers.append(nn.Linear(input_size, layers[i]))
            fc_layers.append(nn.ReLU())
            if reg_layers[i] > 0:
                fc_layers.append(nn.Dropout(reg_layers[i]))
            input_size = layers[i]

        self.fc_layers = nn.Sequential(*fc_layers)
        self.output_layer = nn.Linear(layers[-1], 1)

        self._init_weight()

    def forward(self, user_input, item_input):
        # [batch, num_users, embed_size]
        user_latent = self.user_embedding(user_input)
        item_latent = self.item_embedding(item_input)

        vector = torch.cat([user_latent, item_latent], dim=-1)

        vector = self.fc_layers(vector)

        prediction = self.output_layer(vector)
        return prediction

    def _init_weight(self):
        nn.init.normal_(self.user_embedding.weight, std=0.01)
        nn.init.normal_(self.item_embedding.weight, std=0.01)

In [13]:
def calculate_precision_recall(user_ratings, k, threshold):
    user_ratings.sort(key=lambda x: x[0], reverse=True)

    n_rel = sum(true_r >= threshold for _, true_r in user_ratings)
    n_rec_k = sum(est >= threshold for est, _ in user_ratings[:k])
    n_rel_and_rec_k = sum(
        (true_r >= threshold) and (est >= threshold) for est, true_r in user_ratings[:k]
    )

    precision = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1
    recall = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precision, recall

In [14]:
model = RecSysModel(
    num_users=len(label_enc_uid.classes_),
    num_items=len(label_enc_fid.classes_),
    layers=layers,
    reg_layers=reg_layers
).to(device)

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_func = nn.MSELoss()

model.train()

RESUME = 'allow'
wandb.init(
    project='RecSysNCF',
    resume=RESUME,
    name=str(datetime.datetime.now()),
    config={
        'batch_size': batch_size,
        'layers': layers,
        'reg_layers': reg_layers,
        'learning_rate': learning_rate,
        'epochs': epochs,
        'threshold': threshold,
        'k': k
    }
)

wandb.watch(model)
best_f1_score = float('-inf')

for epoch in range(epochs):
    model.train()
    total_train_loss = 0
    print(f'\nEpoch [{epoch+1}/{epochs}]')

    for i, train_data in enumerate(tqdm(train_loader)):

        uid = train_data['uid'].to(device)
        fid = train_data['fid'].to(device)
        rating = train_data["rating"].to(device)

        optimizer.zero_grad()
        output: torch.Tensor = model(
            uid, fid
        )
        # [batch, 1] -> [batch]
        output = output.squeeze()

        loss = loss_func(output, rating)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
    
    avg_train_loss = total_train_loss / len(train_loader)
    print(f"Average Training Loss for Epoch {epoch+1}: {avg_train_loss:.4f}")

    model.eval()  
    total_val_loss = 0

    y_pred = []
    y_true = []
    user_ratings_comparison = defaultdict(list)

    with torch.no_grad():
        for i, val_data in enumerate(tqdm(val_loader)):
            uid = val_data['uid'].to(device)
            fid = val_data['fid'].to(device)
            rating = val_data["rating"].to(device)

            output = model(
                uid, fid
            )
            output = output.squeeze()

            loss = loss_func(output, rating)
            
            total_val_loss += loss.item()

            y_pred.extend(output.cpu().numpy())
            y_true.extend(rating.cpu().numpy())
            for user, pred, true in zip(uid, output, rating):
                user_ratings_comparison[user.item()].append((pred.item(), true.item()))


    avg_val_loss = total_val_loss / len(val_loader)
    # rmse = root_mean_squared_error(y_true, y_pred)
    avg_precision = 0.0
    avg_recall = 0.0

    for user, user_ratings in user_ratings_comparison.items():
        precision, recall = calculate_precision_recall(user_ratings, k, threshold)
        avg_precision += precision
        avg_recall += recall
    
    avg_precision /= len(user_ratings_comparison)
    avg_recall /= len(user_ratings_comparison)
    f1_score = (2 * avg_precision * avg_recall) / (avg_precision + avg_recall)
    print(f'val_loss: {avg_val_loss:.4f}, precision: {avg_precision:.4f}, recall: {avg_recall:.4f}, f1:{f1_score:.4f}')


    if f1_score > best_f1_score:
        best_f1_score = f1_score
        torch.save(model.state_dict(), f'{CHECKPOINT_PATH}/ncf.pth')
        print(f"f1-score improved. Model checkpoint saved at epoch {epoch+1}.")
    else:
        print("f1-score did not improve.")

    wandb.log({
        'train_loss': avg_train_loss,
        'val_loss': avg_val_loss,
        'precision': avg_precision,
        'recall': avg_recall,
        'f1': f1_score
    })

    print('-' * 50)

wandb.finish()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: thanh309. Use `wandb login --relogin` to force relogin



Epoch [1/40]


100%|██████████| 7225/7225 [01:03<00:00, 114.47it/s]


Average Training Loss for Epoch 1: 8.1619


100%|██████████| 903/903 [01:46<00:00,  8.45it/s]


...

100%|██████████| 7225/7225 [01:04<00:00, 111.93it/s]


Average Training Loss for Epoch 40: 2.5277


100%|██████████| 903/903 [01:44<00:00,  8.61it/s]


val_loss: 2.3960, precision: 0.9055, recall: 0.7043, f1:0.7923
f1-score did not improve.
--------------------------------------------------


f1,▁▃▆▇▇▇▇▇▇█▇▇██████████▇█▇▇▇█▇█▇▇█▇▇▇▇▇▇▇
precision,▇█▃▁▁▂▄▃▄▅▅▅▄▄▄▅▄▅▄▅▅▄▅▄▅▄▄▃▅▃▄▃▃▃▄▄▄▄▄▅
recall,▁▃▆▇▇▇▇█▇▇▇▇███▇█▇████▇█▇▇▇█▇█▇▇█▇▇▇▇▇▇▇
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▄▄▄▄▃▃▃▃▃▂▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▂▁
f1,0.7923
precision,0.90551
recall,0.70426
train_loss,2.52766
val_loss,2.396
